In [ ]:
import pandas as pd
import numpy as np
from itertools import product
import requests

def macd(df, fast_period, slow_period, signal_period):
    df['EMA_fast'] = df['Close'].ewm(span=fast_period, adjust=False).mean()
    df['EMA_slow'] = df['Close'].ewm(span=slow_period, adjust=False).mean()
    df['MACD'] = df['EMA_fast'] - df['EMA_slow']
    df['Signal'] = df['MACD'].ewm(span=signal_period, adjust=False).mean()
    df['MACD_Hist'] = df['MACD'] - df['Signal']
    return df

def bollinger_bands(df, bb_period, num_std_dev):
    df['MA'] = df['Close'].rolling(window=bb_period).mean()
    df['BB_Upper'] = df['MA'] + (df['Close'].rolling(window=bb_period).std() * num_std_dev)
    df['BB_Lower'] = df['MA'] - (df['Close'].rolling(window=bb_period).std() * num_std_dev)
    return df

def atr(df, atr_period):
    df['High-Low'] = df['High'] - df['Low']
    df['High-PrevClose'] = abs(df['High'] - df['Close'].shift(1))
    df['Low-PrevClose'] = abs(df['Low'] - df['Close'].shift(1))
    df['TrueRange'] = df[['High-Low', 'High-PrevClose', 'Low-PrevClose']].max(axis=1)
    df['ATR'] = df['TrueRange'].rolling(window=atr_period).mean()
    return df

def backtest(df, fast_period, slow_period, signal_period, bb_period, num_std_dev, atr_period):
    df = macd(df, fast_period, slow_period, signal_period)
    df = bollinger_bands(df, bb_period, num_std_dev)
    df = atr(df, atr_period)

    initial_capital = 100000
    capital = initial_capital
    position = 0
    returns = []

    for i in range(1, len(df)):
        if df['MACD'][i-1] < df['Signal'][i-1] and df['MACD'][i] > df['Signal'][i] and df['Close'][i] < df['BB_Lower'][i]:
            if position == 0:
                position = capital / df['Close'][i]
                capital = 0
        elif df['MACD'][i-1] > df['Signal'][i-1] and df['MACD'][i] < df['Signal'][i] and df['Close'][i] > df['BB_Upper'][i]:
            if position > 0:
                capital = position * df['Close'][i]
                position = 0

        portfolio_value = capital + position * df['Close'][i]
        returns.append((portfolio_value - initial_capital) / initial_capital)

    total_return = (capital + position * df['Close'].iloc[-1] - initial_capital) / initial_capital
    return total_return

def grid_search(df):
    fast_range = range(5, 21, 2)
    slow_range = range(20, 51, 5)
    signal_range = range(5, 16, 2)
    bb_period_range = range(10, 31, 5)
    num_std_dev_range = np.arange(1, 3.5, 0.5)
    atr_period_range = range(10, 31, 5)

    results = []

    for fast, slow, signal, bb_period, num_std_dev, atr_period in product(fast_range, slow_range, signal_range, bb_period_range, num_std_dev_range, atr_period_range):
        if fast < slow:
            performance = backtest(df, fast, slow, signal, bb_period, num_std_dev, atr_period)
            results.append((fast, slow, signal, bb_period, num_std_dev, atr_period, performance))

    best_params = max(results, key=lambda x: x[6])
    return best_params

def fetch_data(symbol, api_key, output_size='compact'):
    base_url = 'https://www.alphavantage.co/query?'
    function = 'TIME_SERIES_DAILY_ADJUSTED'
    url = f"{base_url}function={function}&symbol={symbol}&apikey={api_key}&outputsize={output_size}&datatype=csv"
    df = pd.read_csv(url)
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df.set_index('timestamp', inplace=True)
    df.sort_index(inplace=True)
    return df

# Example of running the full script
api_key = 'YOUR_ALPHA_VANTAGE_API_KEY'
df = fetch_data('AAPL', api_key)
best_params = grid_search(df)
print(f"Best Parameters: MACD Fast={best_params[0]}, Slow={best_params[1]}, Signal={best_params[2]}, "
      f"BB Period={best_params[3]}, Std Dev={best_params[4]}, ATR Period={best_params[5]}, Performance={best_params[6]}")
